In [1]:
import pandas as pd
import requests as rq
import json
from datetime import datetime as dt
import ast

In [2]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [3]:
market = json.loads(api.text)

In [4]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [14]:
round_num = '15'

In [6]:
roundN = pd.DataFrame(market["atletas"])

In [7]:
with open("../../status_id.json") as file:
    status_id = json.load(file)
with open("../../positions.json") as file:
    pos_id = json.load(file)

In [8]:
roundN = roundN[["rodada_id", "atleta_id", "nome", "apelido", "posicao_id", "clube_id", "scout", "pontos_num", "preco_num", "variacao_num"]]
roundN.columns = ["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "scout", "total_score", "current_price", "price_delta"]

In [9]:
roundN.position = roundN.position.apply(lambda x: pos_id[str(x)])

In [10]:
def scout_parser(scout, col_names):
    scout_dict = ast.literal_eval(scout)
    scout_list = []
    for col in col_names:
        try:
            el = scout_dict[col]
        except:
            el = 0
        scout_list.append(el)
    return pd.DataFrame(columns = col_names, data = [scout_list]).iloc[0]

In [11]:
col_names = ["G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC"]
roundN[col_names] = roundN.scout.apply(lambda x: scout_parser(str(x), col_names))
roundN = roundN[["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price", "price_delta"]]

In [12]:
roundN.head(10)

,round_num,player_id,player_name,player_nickname,position,team_id,G,A,FT,FD,...,DD,RB,GC,CV,CA,GS,FC,total_score,current_price,price_delta
0,14,37646,Ebert William Amâncio,Betão,ZAG,314,0,0,0,0,...,0,18,0,0,4,0,10,1.20,3.89,-0.43
1,14,37246,Ney Franco da Silveira Júnior,Ney Franco,TEC,290,0,0,0,0,...,0,0,0,0,0,0,0,2.55,8.63,0.05
2,14,37662,Jonathan Cícero Moreira,Jonathan,LAT,293,1,0,0,1,...,0,6,0,0,2,0,4,0.00,7.30,0.00
3,14,37694,Henrique Pacheco de Lima,Henrique,MEI,283,0,0,0,2,...,0,26,0,0,0,0,13,5.50,8.43,-0.09
4,14,37701,Leonardo da Silva Moura,Léo Moura,LAT,284,0,1,0,0,...,0,5,0,0,0,0,2,-0.60,5.48,-0.86
5,14,37614,Evandro Goebel,Evandro,MEI,277,0,0,0,0,...,0,2,0,0,0,0,1,0.00,4.67,-0.38
6,14,37656,Fábio Deivson Lopes Maciel,Fábio,GOL,283,0,0,0,0,...,14,0,0,0,0,16,0,-0.50,9.42,0.02
7,14,37673,Alecsandro Barbosa Felisbino,Alecsandro,ATA,341,0,0,0,2,...,0,1,0,0,0,0,6,4.70,4.92,1.10
8,14,37657,Eduardo Luís Abonizio de Souza,Edu Dracena,ZAG,275,0,0,0,1,...,0,4,0,0,0,0,3,0.00,6.97,0.00
9,14,37788,Marcelo Boeck,Marcelo Boeck,GOL,356,0,0,0,0,...,2,0,0,0,1,2,0,0.00,3.31,0.00


In [13]:
roundN.to_csv("2019_round"+ round_num + "_scouts.csv", index=False)

In [15]:
status = roundN[["rodada_id", "atleta_id", "status_id"]]

KeyError: "['rodada_id' 'atleta_id' 'status_id'] not in index"

In [ ]:
status.rodada_id = status.rodada_id + 1
now = dt.now()
status["query_time"] = dt(now.year, now.month, now.day, now.hour, now.minute, now.second)
status.status_id = status.status_id.apply(lambda x: status_id[str(x)])

In [ ]:
status = status[["rodada_id", "query_time", "atleta_id", "status_id"]]
status.columns = ["round_num", "query_time", "player_id", "status_id"]
status.head()

In [23]:
filename = "../status/2019_round"
if status.round_num[0] < 10:
    filename = filename + "0" + str(status.round_num[0]) + "_status.csv"
else:
    filename = filename + str(status.round_num[0]) + "_status.csv"

status.to_csv(filename, index=False)